In [170]:
import pandas as pd 
import datetime as dt 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from scipy.stats import poisson
import math
import random

In [281]:
#df = pd.read_csv('../data/trip_cleaned_final.csv') 
df['pickup_time_act'] = pd.to_datetime(df['pickup_datetime']).dt.time
df['dropoff_time_act'] = pd.to_datetime(df['dropoff_datetime']).dt.time
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

df.shape

(13873809, 39)

In [377]:
m_df = pd.read_csv("../data/prob_match_combined_new.csv")
m_df 
m_df.drop(columns=["Unnamed: 0"])


,taxi_zone,time_stamp,time_interval,arr_weekday,arr_weekend,avg_weekday,avg_weekend,prob_pu_weekday,prob_pu_weekend,c_pu_weekday,...,supply_surplus_weekday,supply_surplus_weekend,vacant_weekday,prob_no_vacant_weekday,vacant_weekend,prob_no_vacant_weekend,prob_match_weekday,prob_match_weekend,mod_prob_match_weekday,mod_prob_match_weekend
0,1,1,00:00:00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,...,True,True,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,2,00:30:00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,...,True,True,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1,3,01:00:00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,...,True,True,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1,4,01:30:00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,...,True,True,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1,5,02:00:00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,...,True,True,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12619,263,44,21:30:00,226.700000,203.938497,226.70,202.7,1.0,1.0,4534.000000,...,True,True,0.364163,0.635837,0.124624,0.875376,0.116969,0.186646,0.635837,0.875376
12620,263,45,22:00:00,240.949479,205.027740,231.80,203.5,1.0,1.0,4818.989575,...,True,True,0.374965,0.625035,0.213479,0.786521,0.124905,0.173218,0.625035,0.786521
12621,263,46,22:30:00,209.819049,186.540169,199.90,185.8,1.0,1.0,4196.380984,...,True,True,0.420148,0.579852,0.235151,0.764849,0.116088,0.158219,0.579852,0.764849
12622,263,47,23:00:00,199.235846,199.689468,191.75,198.7,1.0,1.0,3984.716911,...,True,True,0.449817,0.550183,0.208284,0.791716,0.115821,0.176900,0.550183,0.791716


In [171]:
p_df_org = pd.read_csv("../data/taxi_after_dropoff.csv")
p_df_org = p_df_org.rename(columns={"Unnamed: 0": "zone"})

In [172]:
#Check if all prob for a zone equal to 1
p_df_org.sum(axis = 1) - (p_df_org.index+1) 

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
258    1.0
259    1.0
260    1.0
261    1.0
262    1.0
Length: 263, dtype: float64

In [173]:
#p_df_org
p_df = p_df_org.drop(columns=["zone"])
p_df

,1,2,3,4,5,6,7,8,9,10,...,254,255,256,257,258,259,260,261,262,263
0,0.001487,0.000000,0.000000,0.000929,0.0,0.0,0.000248,0.000000,0.0,0.000062,...,0.000000,0.000248,0.000124,0.000062,0.0,0.000000,0.000124,0.010407,0.000434,0.001487
1,0.000000,0.571429,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.071429,0.000000
2,0.000000,0.000000,0.020638,0.001876,0.0,0.0,0.001876,0.000000,0.0,0.000000,...,0.000000,0.001876,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.022514,0.058161
3,0.000000,0.000000,0.000000,0.299248,0.0,0.0,0.000209,0.000000,0.0,0.000015,...,0.000000,0.000539,0.000718,0.000000,0.0,0.000000,0.000105,0.001062,0.000853,0.001346
4,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.032258,0.000000,0.000000,0.0,0.000000,0.000000,0.032258,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,0.000000,0.000000,0.000000,0.004122,0.0,0.0,0.001649,0.000000,0.0,0.000000,...,0.001649,0.000824,0.000824,0.000000,0.0,0.021434,0.000000,0.001649,0.018137,0.073372
259,0.000069,0.000000,0.000000,0.001922,0.0,0.0,0.035562,0.000069,0.0,0.000343,...,0.000000,0.016751,0.003089,0.000000,0.0,0.000000,0.102156,0.000618,0.006934,0.008856
260,0.000015,0.000000,0.000000,0.001988,0.0,0.0,0.000104,0.000000,0.0,0.000059,...,0.000000,0.000252,0.000208,0.000000,0.0,0.000000,0.000045,0.318062,0.000282,0.000430
261,0.000000,0.000000,0.000000,0.001605,0.0,0.0,0.000507,0.000000,0.0,0.000026,...,0.000000,0.000204,0.000086,0.000000,0.0,0.000000,0.000132,0.000125,0.371068,0.145969


In [174]:

print(p_df.max(axis=1))
print(p_df.idxmax(axis=1))
p_list = list(p_df.iloc[0,:])
max(p_list)
print(p_list.index(max(p_list)) +1  )
p_df.columns


0      0.361767
1      0.571429
2      0.138837
3      0.319128
4      0.161290
         ...   
258    0.088211
259    0.219621
260    0.318062
261    0.371068
262    0.316449
Length: 263, dtype: float64
0      231
1        2
2      138
3       79
4      231
      ... 
258     74
259    138
260    261
261    262
262    263
Length: 263, dtype: object
231


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       ...
       '254', '255', '256', '257', '258', '259', '260', '261', '262', '263'],
      dtype='object', length=263)

In [353]:
# This policy always reposition to highest prob 
def policy_best_zone(zone_in) :
    p_list = list(p_df.iloc[(zone_in-1),:])
    zone_repos = p_list.index(max(p_list))+1
    return(zone_repos)

# This policy always reposition to any prob > 3%
def policy_top_zone(zone_in) :
    zone_s = p_df.iloc[(zone_in-1),:]
    #Pick a destination zone with prob > 5% 
    #print(zone_s[zone_s>0.03].index)
    #print(random.choice(zone_s[zone_s>0.03].index))
    return (random.choice(zone_s[zone_s>0.03].index))

In [351]:
repos_zone = policy_best_zone(140)
print(repos_zone)

repos_zone = policy_top_zone(140)
print(repos_zone)

140
140


In [212]:
trip_df_org = pd.read_csv("../data/dropoff_after_pickup.csv")
#trip_df_org.head()
trip_df = trip_df_org.drop(columns=["Unnamed: 0"])
trip_df.head()

,1,2,3,4,5,6,7,8,9,10,...,254,255,256,257,258,259,260,261,262,263
0,0.728571,0.000000,0.000000,0.00000,0.0,0.0,0.0000,0.00000,0.00000,0.0,...,0.000000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
1,0.000000,0.727273,0.000000,0.00000,0.0,0.0,0.0000,0.00000,0.00000,0.0,...,0.000000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
2,0.000000,0.000000,0.095238,0.00000,0.0,0.0,0.0000,0.00000,0.00000,0.0,...,0.095238,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
3,0.000640,0.000000,0.000020,0.02046,0.0,0.0,0.0042,0.00004,0.00008,0.0,...,0.000060,0.01182,0.01276,0.00062,0.00004,0.0001,0.00064,0.00444,0.00594,0.01074
4,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0000,0.00000,0.00000,0.0,...,0.000000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000


In [266]:
# Code to calculate next zone 

#trip_df.head()
pick_zone = 79
zone_s = trip_df.iloc[(pick_zone-1),:]
#Pick a destination zone with prob > 5% 
print(zone_s[zone_s>0.03].index)
print(random.choice(zone_s[zone_s>0.03].index))
#high_rides = zone_1 > 0.05
#zone_1[high_rides]

Index(['79', '107', '148', '170', '186', '234', '249'], dtype='object')
170


In [179]:
# Check sum of prob = 1.0 
trip_df.sum(axis = 1)

0      1.0
1      1.0
2      1.0
3      1.0
4      0.0
      ... 
258    1.0
259    1.0
260    1.0
261    1.0
262    1.0
Length: 263, dtype: float64

In [180]:
tt_df = pd.read_csv("../data/trip_time_scr_dst_t.csv")
tt_df.shape


(471868, 7)

In [181]:
tt_df.tail()

,pickup_datetime_index,pickup_datetime_interval,pickup_taxizone_id,dropoff_taxizone_id,mean,median,count
471863,47,23:30:00,263.0,257.0,35.833333,34.500000,6
471864,47,23:30:00,263.0,260.0,17.536667,16.983333,15
471865,47,23:30:00,263.0,261.0,25.850000,25.000000,7
471866,47,23:30:00,263.0,262.0,4.028523,3.316667,149
471867,47,23:30:00,263.0,263.0,4.614421,3.000000,193


In [319]:
dist_df = pd.read_csv("../data/trip_distance_scr_dst.csv")

In [320]:
repos_cost = (dist_df.loc[((dist_df["pickup_taxizone_id"] == dest_zone) & (dist_df["dropoff_taxizone_id"] == int(repos_zone))), "mean"].describe().mean())*0.124
        

,pickup_taxizone_id,dropoff_taxizone_id,mean,median,count
0,1.0,1.0,2.732745,0.700,51
1,1.0,43.0,0.500000,0.500,1
2,1.0,50.0,3.030000,3.030,1
3,1.0,70.0,24.700000,24.700,1
4,1.0,79.0,15.570000,15.570,1
...,...,...,...,...,...
28220,263.0,259.0,11.204483,10.955,58
28221,263.0,260.0,5.177234,5.090,282
28222,263.0,261.0,7.713881,7.945,286
28223,263.0,262.0,0.688296,0.600,8057


In [182]:
fare_df = pd.read_csv("../data/fare_amount_scr_dst_t.csv")
fare_df.shape

(471868, 7)

In [183]:
fare_df.head()

,pickup_datetime_index,pickup_datetime_interval,pickup_taxizone_id,dropoff_taxizone_id,mean,median,count
0,0,00:00:00,2.0,2.0,8.666667,6.0,3
1,0,00:00:00,3.0,69.0,24.500000,24.5,1
2,0,00:00:00,4.0,4.0,7.175000,4.5,40
3,0,00:00:00,4.0,7.0,21.305556,20.5,18
4,0,00:00:00,4.0,13.0,14.437500,14.0,16


In [184]:
start_time = 0
start_zone = random.randint(1, 263)

print(start_time, start_zone)

0 216


In [209]:
trip_duration = tt_df.loc[((tt_df["pickup_taxizone_id"] == 34) & (tt_df["dropoff_taxizone_id"] ==35) ), "mean"] #.describe().mean()
print(trip_duration)

Series([], Name: mean, dtype: float64)


In [370]:
# Simulation for shift of 8 hours 
start_time = 0

prob = 0.6
random.seed(50)
cur_ran_zone = random.randint(1, 263) # decide later if we want to fix it for all episodes

earnings_list = []

for i in range(0,10) :
    cur_zone = cur_ran_zone
    print("episode", i, cur_zone)
    cur_time = start_time
    trip_completion = 0
    earnings = 0
    while(cur_time < start_time+17):
    
        #check cur taxi zone and cur_time and find matching prob 
        match_prob = prob 

        if (match_prob >= 0.9 ) :
            pickup_time = random.randint(1, 4)
        elif (match_prob >= 0.7  ):
            pickup_time = random.randint(5, 10)
        elif (match_prob >= 0.5  ):
            pickup_time = random.randint(11, 15)
        elif (match_prob >= 0.3  ):
            pickup_time = random.randint(16, 20)
        elif (match_prob >= 0.1  ):
            pickup_time = random.randint(21, 29)
    
   
    
        # Choose a ride, choose a destination zone 
        zone_s = trip_df.iloc[cur_zone-1,:]
        top_dest_list = list(zone_s[zone_s>0.03].index)
        #print(top_dest_list)
        dest_zone = int(random.choice(top_dest_list))
        #trip_duration = tt_df.loc[((tt_df["pickup_taxizone_id"] == cur_zone) & (tt_df["dropoff_taxizone_id"] ==dest_zone) & (tt_df["pickup_datetime_index"]==cur_time)), "mean"].values
        #print(type(cur_zone), type(dest_zone))
        trip_duration = tt_df.loc[((tt_df["pickup_taxizone_id"] == cur_zone) & (tt_df["dropoff_taxizone_id"] ==dest_zone)), "mean"].describe().mean()
        #trip_duration = 0
        #print(trip_duration)
        trip_fare = fare_df.loc[((fare_df["pickup_taxizone_id"] == cur_zone) & (fare_df["dropoff_taxizone_id"] ==dest_zone)), "mean"].describe().mean()
        fuel_cost = (dist_df.loc[((dist_df["pickup_taxizone_id"] == cur_zone) & (dist_df["dropoff_taxizone_id"] ==dest_zone)), "mean"].describe().mean())*0.124

        earnings = earnings + trip_fare - fuel_cost
        
        trip_completion = trip_completion + pickup_time + trip_duration
        
    
        # call policy now to reposition 
        repos_zone = policy_best_zone(dest_zone)
        repos_cost = (dist_df.loc[((dist_df["pickup_taxizone_id"] == dest_zone) & (dist_df["dropoff_taxizone_id"] == int(repos_zone))), "mean"].describe().mean())*0.124
        repos_time = tt_df.loc[((tt_df["pickup_taxizone_id"] == dest_zone) & (tt_df["dropoff_taxizone_id"] ==repos_zone)), "mean"].describe().mean()
        
        trip_completion = trip_completion + repos_time 
        
        print("repos_cost:, repos_zone ", repos_cost, repos_zone)
    
        print(cur_time, pickup_time, cur_zone, dest_zone, trip_duration, trip_completion, trip_fare, fuel_cost,earnings)
    
        if (trip_completion >= 30 ):
            trip_completion = trip_completion - 30 
            cur_time = cur_time + 1
            
        cur_zone = repos_zone
        earnings = earnings-repos_cost
        
    earnings_list.append(earnings)
    
    

episode 0 255
repos_cost:, repos_zone  0.1497554674220962 80
0 13 255 80 11.329921212509351 36.325216040486794 11.373660854402225 0.15215054568662895 11.221510308715596
repos_cost:, repos_zone  0.11032569972702443 255
1 12 80 255 10.854112847032685 40.01081407469175 11.031851638729288 0.13264023779499187 21.970966242227796
repos_cost:, repos_zone  0.16342081203007514 37
2 13 255 37 16.231602222254622 52.056501354798684 14.513958028203124 0.27795247027741116 36.09664610042648
repos_cost:, repos_zone  0.16342081203007514 37
3 15 37 37 12.814085057852306 62.684671470503304 12.538980977594871 0.16342081203007514 48.3087854539612
repos_cost:, repos_zone  0.11032569972702443 255
4 12 37 255 14.245194639296962 69.76135129697255 12.991358782823738 0.23471487816531275 60.90200854658955
repos_cost:, repos_zone  0.16342081203007514 37
5 15 255 37 16.231602222254622 83.80703857707948 14.513958028203124 0.27795247027741116 75.02768840478824
repos_cost:, repos_zone  0.16342081203007514 37
6 12 37 37

repos_cost:, repos_zone  0.10287175203717264 170
6 11 137 170 10.294078146848255 51.812247436935564 10.382016118940303 0.10313498340224796 81.48344472625753
repos_cost:, repos_zone  0.10358573068412792 79
7 13 170 79 12.057962744579797 56.649198424149844 12.121065311093997 0.20058485089891148 93.30105343441545
repos_cost:, repos_zone  0.12358126297440782 186
8 12 79 186 15.965359542255744 66.35443552585193 14.083045208865945 0.23844341466121693 107.04206949793604
repos_cost:, repos_zone  0.10165868079584854 162
9 11 186 162 15.63116803476094 73.57832416484082 13.50348331371365 0.1965733822868853 120.22539816638839
repos_cost:, repos_zone  0.11135073585152262 164
10 15 162 164 12.557040760136486 82.08216975346502 11.633297484638945 0.1409986500613027 131.61603832017016
repos_cost:, repos_zone  0.1058040890295201 48
11 15 164 48 13.834099350532231 91.3643905865927 12.414318443199889 0.16386000443291285 143.75514602308564
repos_cost:, repos_zone  0.13798622005223132 246
12 12 48 246 11.67

repos_cost:, repos_zone  0.10287175203717264 170
15 11 79 170 12.719532245870077 133.577328388312 12.350799863385728 0.19515131853691642 188.17099029523888
repos_cost:, repos_zone  0.10287175203717264 170
16 14 170 170 10.282147402567082 138.14162319344615 10.60945470694779 0.10287175203717264 198.57470149811232
episode 7 255
repos_cost:, repos_zone  0.14096035101715182 256
0 14 255 256 9.95437541913837 36.34345098259176 10.300418387898565 0.1140695143494248 10.186348873549141
repos_cost:, repos_zone  0.14096035101715182 256
1 11 256 256 12.389075563453389 42.12160210949854 12.035206842908343 0.14096035101715182 21.939635014423178
repos_cost:, repos_zone  0.1497554674220962 80
2 14 256 80 11.312242261139197 49.429139198615175 11.081717551978027 0.15115064489058166 32.729241570493464
repos_cost:, repos_zone  0.14096035101715182 256
3 14 80 256 11.143927010899686 56.96214177296825 11.08755746277533 0.13601948051948046 43.53102408532722
repos_cost:, repos_zone  0.16342081203007514 37
4 15

In [371]:
print(earnings_list)
earnings_avg = sum(earnings_list) / len(earnings_list)
print(earnings_avg)

[205.54399889372542, 191.0811208263522, 198.52235780825256, 203.49568051217386, 200.22555215344147, 192.12421301258027, 198.47182974607514, 195.61159228457237, 189.515037741581, 192.62359206802546]
196.72149750467798


In [373]:
# Simulation for shift of 8 hours 
start_time = 0

prob = 0.6
random.seed(50)
cur_ran_zone = random.randint(1, 263) # decide later if we want to fix it for all episodes

earnings_list = []

for i in range(0,10) :
    cur_zone = int(cur_ran_zone)
    print("episode", i, cur_zone)
    cur_time = start_time
    trip_completion = 0
    earnings = 0
    while(cur_time < start_time+17):
    
        #check cur taxi zone and cur_time and find matching prob 
        match_prob = prob 

        if (match_prob >= 0.9 ) :
            pickup_time = random.randint(1, 4)
        elif (match_prob >= 0.7  ):
            pickup_time = random.randint(5, 10)
        elif (match_prob >= 0.5  ):
            pickup_time = random.randint(11, 15)
        elif (match_prob >= 0.3  ):
            pickup_time = random.randint(16, 20)
        elif (match_prob >= 0.1  ):
            pickup_time = random.randint(21, 29)
    
   
    
        # Choose a ride, choose a destination zone 
        zone_s = trip_df.iloc[cur_zone-1,:]
        top_dest_list = list(zone_s[zone_s>0.03].index)
        #print(top_dest_list)
        dest_zone = int(random.choice(top_dest_list))
        #trip_duration = tt_df.loc[((tt_df["pickup_taxizone_id"] == cur_zone) & (tt_df["dropoff_taxizone_id"] ==dest_zone) & (tt_df["pickup_datetime_index"]==cur_time)), "mean"].values
        #print(type(cur_zone), type(dest_zone))
        trip_duration = tt_df.loc[((tt_df["pickup_taxizone_id"] == cur_zone) & (tt_df["dropoff_taxizone_id"] ==dest_zone)), "mean"].describe().mean()
        #trip_duration = 0
        #print(trip_duration)
        trip_fare = fare_df.loc[((fare_df["pickup_taxizone_id"] == cur_zone) & (fare_df["dropoff_taxizone_id"] ==dest_zone)), "mean"].describe().mean()
        fuel_cost = (dist_df.loc[((dist_df["pickup_taxizone_id"] == cur_zone) & (dist_df["dropoff_taxizone_id"] ==dest_zone)), "mean"].describe().mean())*0.124

        earnings = earnings + trip_fare - fuel_cost
 
        
        trip_completion = trip_completion + pickup_time + trip_duration
        print(cur_time, pickup_time, cur_zone, dest_zone, trip_duration, trip_completion, trip_fare, earnings)
    
        
    
        # call policy now to reposition 
        repos_zone = policy_top_zone(dest_zone)
        repos_cost = (dist_df.loc[((dist_df["pickup_taxizone_id"] == dest_zone) & (dist_df["dropoff_taxizone_id"] == int(repos_zone))), "mean"].describe().mean())*0.124
        repos_time = tt_df.loc[((tt_df["pickup_taxizone_id"] == dest_zone) & (tt_df["dropoff_taxizone_id"] ==repos_zone)), "mean"].describe().mean()
        
        trip_completion = trip_completion + repos_time 
        
        print("repos_cost:, repos_zone ", repos_cost, repos_zone)
    
        if (trip_completion >= 30 ):
            trip_completion = trip_completion - 30 
            cur_time = cur_time + 1
            
        cur_zone = int(repos_zone)
        earnings = earnings-repos_cost
        
    earnings_list.append(earnings)

episode 0 255
0 13 255 80 11.329921212509351 24.32992121250935 11.373660854402225 11.221510308715596
repos_cost:, repos_zone  0.13264023779499187 255
0 12 255 255 10.831485187172277 47.161406399681624 10.904778596584025 21.883322967777605
repos_cost:, repos_zone  0.11032569972702443 255
1 13 255 37 16.231602222254622 46.39300862193625 14.513958028203124 36.009002825976296
repos_cost:, repos_zone  0.39658160469667314 148
2 12 148 79 10.037462884479059 38.4304715064153 10.348081785891212 45.850544629222256
repos_cost:, repos_zone  0.11557843730040403 107
3 13 107 107 10.06175265722956 31.492224163644863 10.330153610157037 55.95811885896947
repos_cost:, repos_zone  0.11784376808239751 113
4 13 113 161 16.247691064757138 30.739915228402 14.425807234875547 70.00872294927076
repos_cost:, repos_zone  0.09773479976750384 162
5 11 162 170 10.805337109484046 22.545252337886048 10.666728100320519 80.45977076916397
repos_cost:, repos_zone  0.10303054952165293 233
5 11 233 107 11.224118373411203 44

repos_cost:, repos_zone  0.14001945284879708 141
2 11 141 161 14.572618219424985 55.383535755627875 12.971561047314388 63.45051633612243
repos_cost:, repos_zone  0.09773479976750384 162
3 14 162 164 12.557040760136486 51.94057651576436 11.633297484638945 74.84508037093258
repos_cost:, repos_zone  0.10904314714736851 234
4 14 234 249 11.060414164112583 47.00099067987695 10.970572962689474 85.56761829937744
repos_cost:, repos_zone  0.11407969076021422 90
5 12 90 48 13.235501491915672 42.23649217179262 12.508981543243493 97.77343468646605
repos_cost:, repos_zone  0.1557346224167196 68
6 11 68 170 14.889926976812259 38.12641914860488 13.232167170969284 110.65567263031629
repos_cost:, repos_zone  0.11211246832678022 137
7 12 137 162 11.253135728839146 31.37955487744403 11.186981622879433 121.58710993283343
repos_cost:, repos_zone  0.15046983290419902 141
8 14 141 141 9.661483858601272 25.0410387360453 10.18887917766315 131.52241255732395
repos_cost:, repos_zone  0.13747301959641597 236
8 14

repos_cost:, repos_zone  0.10296605345366186 90
8 15 90 186 9.85967207528445 44.391165526162325 10.158499875049586 131.61226687487564
repos_cost:, repos_zone  0.12055267167770407 234
9 13 234 230 14.55976839254517 41.95093391870749 13.015966063181786 144.31225750520306
repos_cost:, repos_zone  0.11508857734398101 163
10 15 163 170 13.800541947747947 40.751475866455436 12.536215749789944 156.5615003577272
repos_cost:, repos_zone  0.1183121452154106 162
11 12 162 234 13.216273528509026 35.967749394964464 12.460753378434047 168.7115187788722
repos_cost:, repos_zone  0.11871069811928835 164
12 15 164 163 13.198597376454632 34.1663467714191 12.316419613976107 180.73622049974193
repos_cost:, repos_zone  0.10513258100756892 161
13 11 161 163 10.94472150408337 26.11106827550247 10.629631647924782 191.15732324304764
repos_cost:, repos_zone  0.10863306141275703 163
13 15 163 142 10.71099297407631 51.82206124957878 10.740576172763234 201.66799517240818
repos_cost:, repos_zone  0.10244989010988981

repos_cost:, repos_zone  0.12789334218093407 170
10 13 170 79 12.057962744579797 25.090795665760602 12.121065311093997 160.74907650615265
repos_cost:, repos_zone  0.11025944231261228 113
10 15 113 107 10.679490203965903 50.77028586972651 10.62861474581767 171.1498179207507
repos_cost:, repos_zone  0.12010052276192618 79
11 11 79 234 12.090288722142555 43.86057459186906 11.574041122985479 182.45131630277336
repos_cost:, repos_zone  0.09908853809277995 90
12 13 90 107 11.50426567550519 38.36484026737425 11.074491363742332 193.30174875939613
repos_cost:, repos_zone  0.11784376808239751 113
13 13 113 234 10.665758169220545 32.030598436594794 10.638894228880499 203.70582892042165
repos_cost:, repos_zone  0.10814038900295228 113
14 12 113 161 16.247691064757138 30.27828950135193 14.425807234875547 217.7661363898024
repos_cost:, repos_zone  0.12658873875710464 170
15 14 170 234 10.955772761697729 25.23406226304966 10.91298489379334 228.42136937068454
repos_cost:, repos_zone  0.115727807362815

In [374]:
print(earnings_list)
earnings_avg_2 = sum(earnings_list) / len(earnings_list)
print(earnings_avg_2)

[235.0675589000114, 244.3734385295158, 246.31109744023294, 247.45083799903665, 235.45650001066033, 240.01414779626268, 238.34529295116474, 247.5383200967022, 234.99271579247292, 251.28202694383006]
242.083193645989


In [403]:
cur_zone = 263
time_stamp = 48
match_prob = m_df.loc[((m_df["taxi_zone"]==cur_zone) & (m_df["time_stamp"]==time_stamp)),:]["mod_prob_match_weekday"].values
print(type(match_prob))
print(match_prob)

#[((m_df["taxi_zone"]==cur_zone)&(m_df["taxi_stamd"]==cur_time)),"mod_prob_match_weekday"]

<class 'numpy.ndarray'>
[0.58738502]


### Modified simulator with prob matching lookup

In [414]:
# Simulation for shift of 8 hours 
start_time = 0

#prob = 0.6
random.seed(50)
cur_ran_zone = random.randint(1, 263) # decide later if we want to fix it for all episodes
weekday = 1 #  1 or 0 

earnings_list = []

for i in range(0,10) :
    cur_zone = cur_ran_zone
    print("episode", i, cur_zone)
    cur_time = start_time
    trip_completion = 0
    earnings = 0
    while(cur_time < start_time+17):
    
        #check cur taxi zone and cur_time and find matching prob 
        #match_prob = prob 
        #match_prob = m_df.loc[((m_df["taxi_zone"]==cur_zone)&(m_df["taxi_stamd"]==cur_time)),]
        if (weekday==1) :
            match_prob = m_df.loc[((m_df["taxi_zone"]==cur_zone) & (m_df["time_stamp"]==time_stamp)),:]["mod_prob_match_weekday"].values
        else :
            match_prob = m_df.loc[((m_df["taxi_zone"]==cur_zone) & (m_df["time_stamp"]==time_stamp)),:]["mod_prob_match_weekend"].values
       
        if (match_prob >= 0.9 ) :
            pickup_time = random.randint(1, 4)
        elif (match_prob >= 0.7  ):
            pickup_time = random.randint(5, 10)
        elif (match_prob >= 0.5  ):
            pickup_time = random.randint(11, 15)
        elif (match_prob >= 0.3  ):
            pickup_time = random.randint(16, 20)
        elif (match_prob >= 0.1  ):
            pickup_time = random.randint(21, 29)
    
   
    
        # Choose a ride, choose a destination zone 
        zone_s = trip_df.iloc[cur_zone-1,:]
        top_dest_list = list(zone_s[zone_s>0.03].index)
        #print(top_dest_list)
        dest_zone = int(random.choice(top_dest_list))
        #trip_duration = tt_df.loc[((tt_df["pickup_taxizone_id"] == cur_zone) & (tt_df["dropoff_taxizone_id"] ==dest_zone) & (tt_df["pickup_datetime_index"]==cur_time)), "mean"].values
        #print(type(cur_zone), type(dest_zone))
        trip_duration = tt_df.loc[((tt_df["pickup_taxizone_id"] == cur_zone) & (tt_df["dropoff_taxizone_id"] ==dest_zone)), "mean"].describe().mean()
        #trip_duration = 0
        #print(trip_duration)
        trip_fare = fare_df.loc[((fare_df["pickup_taxizone_id"] == cur_zone) & (fare_df["dropoff_taxizone_id"] ==dest_zone)), "mean"].describe().mean()
        fuel_cost = (dist_df.loc[((dist_df["pickup_taxizone_id"] == cur_zone) & (dist_df["dropoff_taxizone_id"] ==dest_zone)), "mean"].describe().mean())*0.124

        earnings = earnings + trip_fare - fuel_cost
        
        trip_completion = trip_completion + pickup_time + trip_duration
        
    
        # call policy now to reposition 
        #repos_zone = policy_best_zone(dest_zone)
        repos_zone = policy_top_zone(dest_zone)
        repos_cost = (dist_df.loc[((dist_df["pickup_taxizone_id"] == dest_zone) & (dist_df["dropoff_taxizone_id"] == int(repos_zone))), "mean"].describe().mean())*0.124
        repos_time = tt_df.loc[((tt_df["pickup_taxizone_id"] == dest_zone) & (tt_df["dropoff_taxizone_id"] ==repos_zone)), "mean"].describe().mean()
        
        trip_completion = trip_completion + repos_time 
        
        print("repos_cost:, repos_zone ", repos_cost, repos_zone)
    
        print(cur_time, pickup_time, cur_zone, dest_zone, trip_duration, trip_completion, trip_fare, fuel_cost,earnings)
    
        if (trip_completion >= 30 ):
            trip_completion = trip_completion - 30 
            cur_time = cur_time + 1
            
        cur_zone = repos_zone
        earnings = earnings-repos_cost
        
    earnings_list.append(earnings)

    
print(earnings_list)
earnings_avg = sum(earnings_list) / len(earnings_list)
print("Average Earnings :" , earnings_avg)
    

episode 0 255
repos_cost:, repos_zone  0.13264023779499187 255
0 3 255 80 11.329921212509351 14.329921212509351 11.373660854402225 0.15215054568662895 11.221510308715596


/Users/anita/opt/anaconda3/envs/DiDi/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/Users/anita/opt/anaconda3/envs/DiDi/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/Users/anita/opt/anaconda3/envs/DiDi/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/Users/anita/opt/anaconda3/envs/DiDi/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: The truth value of an empty array is ambiguous. Returni

TypeError: unsupported operand type(s) for -: 'str' and 'int'